<h1>Import necessary libraries

In [1]:
import pandas as pd
import psycopg2 as pg
import pandas.io.sql as psql

<h1>Connect to DB

In [2]:
conn = pg.connect(" dbname='dev' host='njoy-testing-env.c2ag7j9q24ns.us-east-1.rds.amazonaws.com' user='reader' port='5416' password='MbmeL8ku6x'")
df = psql.read_sql('SELECT count(1) from public.orders', conn)
df.head()

,count
0,201570


In [3]:
df = psql.read_sql('SELECT * from public.orders', conn)

<h1>Preview of DataFrame

In [4]:
df.head(5)

,order_id,customer_id,created_at,updated_at,closed_at,processed_at,status,number,total_price,total_less_tax,total_tax,payment_status
0,8.974965e+11,230824c2,2019-03-13 18:14:55,2019-03-13 22:01:40,2019-03-13 22:01:39,2019-03-13 18:14:55.000000,fulfilled,1029702,18.28,17.0,1.28,paid
1,8.975937e+11,7100545b,2019-03-13 19:33:50,2019-03-13 23:01:21,2019-03-13 23:01:21,2019-03-13 19:33:50.000000,fulfilled,1029768,9.00,9.0,0.00,paid
2,8.975785e+11,3de1dc29,2019-03-13 19:20:05,2019-03-13 23:01:33,2019-03-13 23:01:33,2019-03-13 19:20:05.000000,fulfilled,1029757,18.02,17.0,1.02,paid
3,8.976117e+11,cb5e4189,2019-03-13 19:50:09,2019-03-13 23:01:12,2019-03-13 23:01:11,2019-03-13 19:50:09.000000,fulfilled,1029780,9.63,9.0,0.63,paid
4,8.976406e+11,579c3141,2019-03-13 20:17:01,2019-03-14 00:01:23,2019-03-14 00:01:23,2019-03-13 20:17:01.000000,fulfilled,1029803,9.26,9.0,0.26,paid


In [5]:
df.shape

(201570, 12)

<h1>Query 1</h1>
<h6>We need basic insight into hourly sales performance. We'd like a query that shows the following metrics by hour: <br>
revenue (total_price less total_tax)<br>
total number of orders<br>
number of orders placed by a customer who has previously placed an order<br>

To make this task easier, I create a column that seperates the hour that each order closed at

In [6]:
df['closed_hour'] = df['closed_at'].dt.hour

In [7]:
#get initial hourly performance
#this groupby gives the hourly revenue and total amount of orders
hourly_orders = df.groupby('closed_hour')['total_less_tax'].agg(['sum','count']).reset_index()

In [8]:
#this groupby gives each customers orders grouped together
customer_counts = df.groupby('customer_id').count().reset_index().sort_values('order_id', ascending = False)

There are a total of 181,105 customers in the database

In [9]:
customer_counts.shape

(181105, 13)

Of the 181,105 customers, 19,339 have made more than 1 purchase shown below

In [10]:
returning_customers = customer_counts.loc[customer_counts['order_id'] > 1][['customer_id', 'order_id']]

In [11]:
returning_customers.shape

(19339, 2)

This function counts the specific hours that each returning customer made their purchases. There were many nan values, so I used a try and except feature to only count values that are integers.

In [14]:
def find_hours_of_returning_customers():
    #create empty dictionary to hold hourly counts
    returning_customer_hours = {}
    
    #use list of returning customers
    for customer in returning_customers.customer_id.to_list():
        #locate each customers individual orders
        order_hours = df.loc[df['customer_id'] == customer].closed_hour.to_list()
        for i in order_hours:
            try:
                #for each purchase, check if the hour is in the dictionary, if not add it
                if int(i) not in returning_customer_hours.keys():
                    returning_customer_hours.update({i: 1})
                    
                #if the hour is already in the dictionary, increment by 1
                else:
                    returning_customer_hours[i] += 1
                    
            #if the value is not an integer, do not count it
            except:
                pass
    return returning_customer_hours

In [15]:
total_returning_per_hour = find_hours_of_returning_customers()

After finding the hourly totals from returning customers, I make a dataframe from the dictionary, then sort it by hour.

In [16]:
returning_df = pd.DataFrame(total_returning_per_hour, index = ['number']).transpose().reset_index()
returning_df_sorted = returning_df.sort_values('index')

Finally, merge the new dataframe with the existing hourly_orders dataframe.

In [17]:
final_orders = pd.merge(hourly_orders, returning_df_sorted, left_on='closed_hour', right_on='index')
final_orders = final_orders.drop(['index'], axis = 1)
final_orders.columns = ['hour', 'total_revenue', 'num_orders', 'num_returning_customers']

Final results for query 1

In [18]:
final_orders

,hour,total_revenue,num_orders,num_returning_customers
0,0.0,54508.00,4481,585
1,1.0,62859.27,5495,706
2,13.0,137.00,6,3
3,14.0,43279.00,3973,241
4,15.0,18872.50,1838,148
5,16.0,185631.40,18287,2881
6,17.0,438159.20,45940,10063
7,18.0,522385.90,54200,12533
8,19.0,233999.00,23461,4219
9,20.0,152570.80,14325,3369


<h1>Query 2</h1>
<h6>Upon further inspection, it has been noticed that there are a number of hours for which there are no orders. It's not clear why, so we'd like a query that lists all hours for which there are no orders.


In [19]:
hours_with_orders = df.groupby(df.closed_hour).count().reset_index()['closed_hour'].to_list()

Build out a simple function to check which hours are not included in hours_with_orders

In [20]:
def find_hours_without_orders(hourly_orders):
    all_hours = list(range(0,24))
    
    #create empty list to hold hours without orders
    hours_without_orders = []
    
    #check each hour to see if it has orders
    for hour in all_hours:
        if hour not in hourly_orders:
            hours_without_orders.append(hour)
    
    return hours_without_orders

In [21]:
hours_without_orders = find_hours_without_orders(hours_with_orders)
no_orders = pd.DataFrame(hours_without_orders)
no_orders.columns = ['hour']

Final results for query 2

In [22]:
no_orders

,hour
0,2
1,3
2,4
3,5
4,6
5,7
6,8
7,9
8,10
9,11


<h1>Query 3</h1>
<h6>Your query from #1 may not have accounted for hours for which there are no orders. Copy your query from #1 and ammend the query to show "0" for all metrics when there is no data for a given hour.

For this query, I just need to join the two dataframes from query 1 and query 2

In [23]:
all_hourly_orders = pd.concat([final_orders, no_orders], join = 'outer')

/Users/austinkrause/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [24]:
all_hourly_orders = all_hourly_orders.sort_values('hour')

Final results for query 3

In [25]:
all_hourly_orders

,hour,num_orders,num_returning_customers,total_revenue
0,0.0,4481.0,585.0,54508.00
1,1.0,5495.0,706.0,62859.27
0,2.0,NaN,NaN,NaN
1,3.0,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN
3,5.0,NaN,NaN,NaN
4,6.0,NaN,NaN,NaN
5,7.0,NaN,NaN,NaN
6,8.0,NaN,NaN,NaN
7,9.0,NaN,NaN,NaN


<h1>Query 4</h1>
<h6>We'd like to know the difference in revenue from customers who have completed one order versus customers who have completed more than one order. Write a query that shows the total revenue (total_price less total_tax) for each of these two groups: New Customers and Returning Customers.

For this query, I split the initial dataframe into two seperate dataframes (new customers and returning customers)

Find the total revenue from each customer and their total amount of orders

In [26]:
customer_totals = df.groupby('customer_id')['total_less_tax'].agg(['sum', 'count']).reset_index()

In [27]:
customer_totals = customer_totals.sort_values('count')

In [28]:
customer_totals.columns = ['customer_id', 'total_revenue', 'num_orders']

Split the grouped dataframe into 2 dataframes

In [29]:
new_customers = customer_totals.loc[customer_totals['num_orders'] == 1]
returning_customers = customer_totals.loc[customer_totals['num_orders'] != 1]

Final results for query 4

In [30]:
new_customers['total_revenue'].sum()

1634850.9699999997

In [31]:
returning_customers['total_revenue'].sum()

397131.3

<h1>Query 5</h1>
<h6>We'd like a rudimentary estimate of when a customer might place another order and the next order's value, based on the time between their last two orders. For all customers that have completed more than one order, write a query that shows the customer_id, last_order_date and next_projected_order_date, and next_projected_order_value where next_projected_order_date is their last order plus the amount of time between their last two orders and next_projected_order_value is the average of their last two orders' total_price less total_tax.

For this query, I can build off of query 4, using the returning customers dataframe

In [32]:
#create list of customers who have > 1 order
returning_list = returning_customers.customer_id.to_list()

In [33]:
#create a new full dataframe using the customers in returning_list
returning_full_df = df[df['customer_id'].isin(returning_list)]

In [34]:
#sort by customer_id and closing time
returning_full_df = returning_full_df.sort_values(['customer_id', 'closed_at'])

In [35]:
returning_full_df.head()

,order_id,customer_id,created_at,updated_at,closed_at,processed_at,status,number,total_price,total_less_tax,total_tax,payment_status,closed_hour
72589,8.561098e+11,0005aaf0,2019-01-05 11:33:56,2019-01-07 17:06:09,2019-01-07 17:06:09,2019-01-05 11:33:56,fulfilled,875783,9.00,9.0,0.00,paid,17.0
72550,8.561096e+11,0005aaf0,2019-01-05 11:33:46,2019-01-07 17:06:10,2019-01-07 17:06:10,2019-01-05 11:33:46,fulfilled,875781,9.00,9.0,0.00,paid,17.0
17935,8.921384e+11,0005aaf0,2019-03-09 11:28:13,2019-03-11 16:20:44,2019-03-11 16:20:44,2019-03-09 11:28:13.000000,fulfilled,1008794,9.00,9.0,0.00,paid,16.0
12222,8.921383e+11,0005aaf0,2019-03-09 11:27:59,2019-03-11 16:20:46,2019-03-11 16:20:46,2019-03-09 11:27:59.000000,fulfilled,1008792,9.00,9.0,0.00,paid,16.0
137103,8.622300e+11,000b5078,2019-02-14 11:16:30,2019-02-14 18:23:00,2019-02-14 18:23:00,2019-02-14 11:16:30.000000,fulfilled,895204,9.54,9.0,0.54,paid,18.0


Create a function to project the next purchase date and amounts

In [36]:
def project_next_purchase(list_of_customers):
    #create empty list to hold dictionaries
    projected_purchases = []
    #set a counter for the customer index
    customer_index = 0
    
    for i in list_of_customers:
        #create individual dataframe for each customer in the loop
        customer_df = returning_full_df.loc[returning_full_df['customer_id'] == i]
        
        #create empty dictionary to hold new projections
        next_purchase_info = {}
        #customer_id
        next_purchase_info['customer_id'] = i
        #most recent order date
        next_purchase_info['last_order_date'] = customer_df['closed_at'].iloc[-1]
        #find time difference between last two orders, then add it to the most recent order
        next_purchase_info['next_projected_order_date'] = customer_df['closed_at'].iloc[-1] + (customer_df['closed_at'].iloc[-1] - customer_df['closed_at'].iloc[-2])
        #find average revenue from last two orders
        next_purchase_info['next_projected_order_value'] = (customer_df['total_less_tax'].iloc[-2] + customer_df['total_less_tax'].iloc[-1])/2
        #append dictionary to list
        projected_purchases.append(next_purchase_info)
        
        customer_index += 1
        
    return projected_purchases

In [37]:
projected_purchases = project_next_purchase(returning_list)

In [38]:
projected_purchases_df = pd.DataFrame.from_dict(projected_purchases)

Preview of final results for query 5. To view more rows, change the value in .head( ).

In [39]:
projected_purchases_df.head(10)

,customer_id,last_order_date,next_projected_order_date,next_projected_order_value
0,7147f69d,2019-03-04 20:05:04,2019-05-06 21:31:46,9.0
1,00286940,2019-03-11 18:31:57,2019-03-22 17:10:30,9.0
2,87885c02,2019-03-04 18:26:55,2019-05-06 18:47:29,9.0
3,f5653a6d,NaT,NaT,9.0
4,78e87a63,2019-01-01 00:02:46,2019-01-01 03:51:38,8.5
5,72436c76,2019-03-06 20:02:13,2019-05-08 19:39:53,9.0
6,0745389a,2019-03-06 19:03:00,2019-05-08 17:54:15,9.0
7,ffcf3677,2019-03-04 18:49:44,2019-05-06 19:47:10,9.0
8,8786ddf4,2019-03-14 17:22:53,2019-04-12 16:26:27,9.0
9,878940f1,2019-03-11 17:08:42,2019-04-08 16:50:32,13.5


<h1>Query 6</h1>
<h6>You might find that some customers have orders that are only seconds apart from each other. This seems like bad data. Copy the query from #5 and ammend to filter out any customers whose orders are less than a minute from each other.

For this query I build on the dataframe from query 5 

In [40]:
#create new column showing the amount of time between the last two orders
projected_purchases_df['time_between_orders'] = projected_purchases_df['next_projected_order_date'] - projected_purchases_df['last_order_date']

Create new dataframe filtering out rows that have < 60 seconds difference between orders

In [41]:
projected_purchases_final = projected_purchases_df[projected_purchases_df['time_between_orders'].dt.total_seconds() > 60]

In [42]:
projected_purchases_final = projected_purchases_final.sort_values('time_between_orders')

Preview of final results for query 6. To view more rows, change the value in .head( ).

In [43]:
projected_purchases_final.head(10)

,customer_id,last_order_date,next_projected_order_date,next_projected_order_value,time_between_orders
14843,8e26e426,2019-03-04 20:10:53,2019-03-04 20:11:59,9.0,00:01:06
6564,4f51ef15,2019-03-08 17:06:05,2019-03-08 17:07:16,9.0,00:01:11
14945,1772e835,2018-12-31 16:31:04,2018-12-31 16:32:20,8.5,00:01:16
8237,554f34be,2019-03-11 16:06:44,2019-03-11 16:08:01,9.0,00:01:17
18646,30a712a2,2019-03-14 18:03:34,2019-03-14 18:04:52,12.5,00:01:18
12189,63994e3b,2019-01-14 17:03:40,2019-01-14 17:05:00,8.5,00:01:20
12213,e1aae39d,2019-02-25 16:23:55,2019-02-25 16:25:16,17.0,00:01:21
8110,cd23d4dc,2019-03-05 19:04:31,2019-03-05 19:05:59,8.0,00:01:28
14687,8d2939cb,2019-02-28 20:06:57,2019-02-28 20:08:27,9.0,00:01:30
8382,54e03148,2019-02-15 17:18:18,2019-02-15 17:19:52,9.0,00:01:34
